In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, UpSampling2D, concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import numpy as np
import cv2
!pip install mat73


Defaulting to user installation because normal site-packages is not writeable


In [2]:
import urllib.request
from tqdm import tqdm

url = 'http://horatio.cs.nyu.edu/mit/silberman/nyu_depth_v2/nyu_depth_v2_labeled.mat'
filename = 'nyu_depth_v2_labeled.mat'

# Get the file size
file_size = int(urllib.request.urlopen(url).info().get('Content-Length', -1))

# Download the file with progress bar
with tqdm(unit='B', unit_scale=True, unit_divisor=1024, miniters=1, desc=filename, total=file_size) as pbar:
    urllib.request.urlretrieve(url, filename, reporthook=lambda blocks, block_size, total_size: pbar.update(block_size))


nyu_depth_v2_labeled.mat:  60%|██████    | 1.66G/2.77G [14:43<05:48, 3.40MB/s]   

: 

: 

In [ ]:
import numpy as np
import h5py

# Load the data from the .mat file using h5py
with h5py.File('nyu_depth_v2_labeled.mat', 'r') as f:
    X = np.array(f['images'])
    y = np.array(f['depths'])

# Save the training data as NumPy arrays
np.save('nyudepth_v2_x_train.npy', X)
np.save('nyudepth_v2_y_train.npy', y)


X = X.transpose((0, 2, 3, 1))

In [ ]:
X.shape,y.shape

In [ ]:
# resize the images to reduce memory usage
X_resized = np.zeros((X.shape[0], 320, 160, 3))
y_resized = np.zeros((y.shape[0], 320, 160))

for i in range(X.shape[0]):
    X_resized[i] = cv2.resize(X[i].transpose(1, 0, 2)[::-1, ...], (160, 320))
    y_resized[i] = cv2.resize(y[i].transpose(1, 0)[::-1, ...], (160, 320))



In [ ]:
del X
del y

In [ ]:
X_resized.shape,y_resized.shape

In [ ]:
pip install scikit-learn


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_resized, y_resized, test_size=0.2, random_state=42)

print(X_train.shape,y_train.shape)
print(X_val.shape,y_val.shape)


In [ ]:
del X_resized
del y_resized

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# define a function to visualize an image with its corresponding depth map
def visualize_image_and_depth_map(X, y):
    # choose a random index
    idx = np.random.randint(X.shape[0])
    # get the image and depth map at that index
    image = X[idx]
    depth_map = y[idx]
    # normalize the pixel values to be between 0 and 1
    image = image / 255.0
    depth_map = depth_map / np.max(depth_map)
    # plot the image and depth map side by side
    fig, ax = plt.subplots(1, 2, figsize=(10, 5))
    ax[0].imshow(image)
    ax[0].set_title('Image')
    ax[1].imshow(depth_map, cmap='gray')
    ax[1].set_title('Depth Map')
    plt.show()


# visualize 4 random images from X_train
for i in range(4):
    visualize_image_and_depth_map(X_train, y_train)

# visualize 4 random images from X_val
for i in range(4):
    visualize_image_and_depth_map(X_val, y_val)


In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt


# Define the model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=X_train.shape[1:]))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(1))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_absolute_error'])

# Train the model
history = model.fit(X_train, y_train, batch_size=8, epochs=10, validation_data=(X_val, y_val))

# Plot the training and validation loss for each epoch
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()


KeyboardInterrupt: 